<div style="text-align: right"> Kalle Alaluusua </div>
<div style="text-align: right"> kalle.alaluusua@aalto.fi </div>


## Simplex and Dual Simplex algorithms
#### with an option to apply Bartels-Golub method for basis updates and Bland's rule to prevent cycling


In [1]:
using LinearAlgebra

function Simplex(A, b, c, βi; Bland::Bool=false, Revised::Bool=true, Print::Bool=true, ϵ::Float64=10^(-6))
    # An implementation of Simplex algorithm with an option to apply Revised or Naive basis updates
    # and Bland's rule to prevent cycling

    # LP dimensions
    n = length(b)
    m = length(c)

    β_initial = copy(βi)
    β = copy(βi)
    x = zeros(m)
    isOptimal = false
    max_itr = 1000
    itr = 1
    
    if Revised
        # LU-factorization of the basis
        # p is the row permutation of A such that L*U = A[p,:]
        B = A[:,β_initial]
        L,U,p = lu(B)
        P = Matrix{Float64}(I, n, n)[p,:]  #Permutation matrix correponding to p
    end
    # Iterate if optimum has not been reached and we have not cycled back to the initial basis
    while (!isOptimal && !(β == β_initial && itr > 1) && itr <= max_itr)
        
        if Revised
            L_inv = inv(L)
            U_inv = inv(U)
            # Basic variables (solution of Bx_β = LUx_β = b)
            x_β = U_inv*(L_inv*(P*b))
            # The price vector (solution of y'B = LU = c)
            y = P*(L_inv'*(U_inv'*(c[β])))
        else
            # The basis
            B = A[:,β]
            B_inv = inv(B)
            # Basic variables (solution of Bx_β = b)
            x_β = B_inv * b
            # The price vector (solution of y'B = c)
            y = B_inv'*c[β]
        end
        
        #The initial solution
        x = zeros(m)
        for (j,x_j)=zip(β, x_β)
            x[j] = x_j
        end
        
        # The reduced costs
        c_hat = c' - y'*A
        # Test for optimal solution by checking if all reduced costs are positive, i.e. !(c_hat[i] < 0) for any i
        isOptimal = !any(c_hat .< 0 - ϵ)

        if isOptimal
            if Print
                println("Optimum reached!")
                println("    The optimal solution: ", x)
                println("    The optimal cost: ", c'*x)
            end
        else
            # Index of the variable with the largest reduced cost, i.e. the entering variable
            if !Bland
                # The index of the variable with the largest reduced cost
                i = argmin(c_hat[1,:]) # c_hat is a row vector
            else
                # Among all variables xⱼ with negative reduced cost, choose the one with the
                # smallest index j as the entering variable
                i = min(findall(c_hat[1,:] .< 0 - ϵ)...)
            end
            
            # Basic direction corresponding to the smallest reduced cost
            if Revised
                d = -U_inv*(L_inv*(P*A[:,i]))
            else
                d = -B_inv * A[:,i]
            end
            
            # As x_i must increase and other nonbasic variable x4 remain zero, this leads to
            d_i = zeros(m)
            d_i[i] = 1
            for (j,d_ij)=zip(β, d)
                d_i[j] = d_ij
            end

            # Step size to the direction d_i
            θ = -x[β]./d
            θ[d.>=0] .= Inf  # d_B(i) > 0 not accepted
            # The minimal ratio
            θ_j = min(θ...)

            # As θ_j only contains elements corresponding to the basic variables,
            # both strategies correct the index of the leaving variable
            # to consider all variables -> x_j is the leaving variable
            if !Bland
                # Index of the leaving variable corresponds to the constraint with min row number
                j_β = argmin(θ)
                j = β[j_β]
            else
                # Among the basic variables that share the minimal ratio,
                # choose the one with smallest variable index as the leaving variable
                j = min(β[findall(θ .== θ_j)]...)
                j_β = findall(β .== j)
            end

            if Revised
                # Apply Bartels-Golub method
                u = L_inv*(P'*(A[:,i]))
                U[:,j_β] = u
            end
            
            # Basic variable indices
            β[β .== j] .= i
            
            # Print results
            if Print
                println("Iteration ",itr)
                println("    Reduced costs: ", c_hat)
                println("    Entering variable index: ", i)
                #println("    Direction d: ", d_i)
                println("    Min ratio test x_β/d: ", θ)
                #println("    Step size θ: ", θ_j)
                println("    Leaving variable index: ", j)
                println("    Basic variable indices: ", β)
            end
            
            itr += 1
        end
    end    
    
    if (β == β_initial && itr > 1)
        if Print
            println("Optimum not reached.\n    Simplex cycled back to the original basic variable indices ", β, ".")
        end
    end
    
    # Return the final solution
    return x
end;


In [2]:
using LinearAlgebra

function DualSimplex(A, b, c, βi; Bland::Bool=false, Revised::Bool=false, Print::Bool=true, ϵ::Float64=10^(-6))
    # An implementation of Dual Simplex algorithm with an option to apply Revised or Naive basis updates
    # and Bland's rule to prevent cycling

    # LP dimensions
    n = length(b)
    m = length(c)

    β_initial = copy(βi)
    β = copy(βi)
    isOptimal = false
    max_itr = 1000
    itr = 1
    
    if Revised
        # LU-factorization of the basis
        # p is the row permutation of A such that L*U = A[p,:]
        B = A[:,β_initial]
        L,U,p = lu(B)
        P = Matrix{Float64}(I, n, n)[p,:]  #Permutation matrix correponding to p
    end
    
    # Iterate if optimum has not been reached and we have not cycled back to the initial basis
    while (!isOptimal && !(β == β_initial && itr > 1) && itr <= max_itr)
        
        # The indices of nonbasic variables
        n = collect(1:length(c))[setdiff(1:end, β)]
        
        if Revised
            L_inv = inv(L)
            U_inv = inv(U)
            # Basic variables (solution of Bx_β = LUx_β = b)
            x_β = U_inv*(L_inv*(P*b))
            # Update coefficient matrix for min ratio test
            A_n = U_inv*(L_inv*(P*A[:, n]))
            # The price vector (solution of y'B = LU = c)
            y = P*(L_inv'*(U_inv'*(c[β])))
        else
            # The basis
            B = A[:,β]
            B_inv = inv(B)
            # Update coefficient matrix for min ratio test
            A_n = B_inv*A[:, n]
            # Basic variables
            x_β = B_inv*b
            # The price vector (solution of y'B = c)
            y = B_inv'*c[β]
        end
        
        #The initial solution
        x = zeros(length(c))
        for (j,x_j)=zip(β, x_β)
            x[j] = x_j
        end
        
        # The reduced costs
        c_hat = c' - y'*A
        # Test for optimal solution by checking if the primal is feasible, i.e. !(x_β[i] < 0) for any i
        isOptimal = !any(x_β .< 0 - ϵ)

        if isOptimal
            if Print
                println("Optimum reached!")
                println("    The optimal primal solution: ", x)
                # Only supports dual solution if A[:,β_initial] is a diagonal matrix
                if Diagonal(A[:,β_initial]) == A[:,β_initial]
                    p = (-c_hat[β_initial]+c[β_initial])./diag(A[:,β_initial])
                    println("    The optimal dual solution: ", p)
                end
                println("    The optimal cost: ", c'*x)
            end
        else
            # Among all variables xⱼ<0, choose the one with the
            # smallest index j as the leaving variable
            i = argmin(x) # c_hat is a row vector
            # The tabular row corresponding to i
            i_β = findall(β .== i)[1]

            isInfeasibleP = !any(A_n[i_β,:] .< 0 - ϵ)
            if isInfeasibleP
                if Print
                    println("The primal is infeasible.")
                end
                return(β)
            end
            
            # The dual is feasible only if the primal optimality condition holds
            isInfeasibleD = any(c_hat .< 0)
            if isInfeasibleD
                if Print
                    println("The dual is infeasible.")
                end
                return(β)
            end
            
            # Minimum ratio test
            r = c_hat[n]./(-A_n[i_β,:])
            r[A_n[i_β,:].>=0] .= Inf  # d_B(i) > 0 not accepted
            r_s = min(r...)
            
            # As r_s only contains elements corresponding to the nonbasic variables,
            # both strategies correct the index of the leaving variable
            # to consider all variables -> x_s is the entering variable
            if !Bland
                # Index of the entering variable corresponds to the constraint with min row number
                s_n = argmin(r)
                s = n[s_n]
            else
                # Among the basic variables that share the minimal ratio,
                # choose the one with smallest variable index as the entering variable
                s = min(n[findall(r .== r_s)]...)
                s_n = findall(n .== s)
            end

            # Basic direction corresponding to column s
            if Revised
                d = -U_inv*(L_inv*(P*A[:,s]))
            else
                d = -B_inv * A[:,s]
            end

            # As x_i must increase and other nonbasic variable x4 remain zero, this leads to
            d_s = zeros(length(c))
            d_s[s] = 1
            for (j,d_sj)=zip(β, d)
                d_s[j] = d_sj
            end

            # Step size to the direction d_s
            θ = (-x./d_s)[i]

            if Revised
                # Apply Bartels-Golub method
                u = L_inv*(P'*(A[:,s_n]))
                U[:,i_β] = u
            end
            
            # Basic variable indices
            β[β .== i] .= s
            
            if Print
                # Print results
                println("Iteration ",itr)
                println("    Current solution: ", x)
                println("    Leaving variable index: ", i)
                #println("    Reduced costs cₙ: ", c_hat[n])
                #println("    -Aₙᵢ: ", -A_n[i_β,:])
                println("    Min ratio test cₙ/-aₙ: ", c_hat[n], " / ", -A_n[i_β,:])
                #println("    Step size θ: ", θ_j)
                println("    Entering variable index: ", s)
                println("    Basic variable indices: ", β)
                #println("    Solution: ", x_new)
                #println("    Cost: ", c_new)
            end
            
            itr += 1
        end
    end    
    
    if (β == β_initial && itr > 1)
        if Print
            println("Optimum not reached.\n    Simplex cycled back to the original basic variable indices ", β, ".")
        end
    end
    
    # Return the final basic variable indices
    return(β)
end;


### Test problem for the anti-cycling functionality of Simplex using Bland's rule

In [3]:
# A will contain the coefficients of the constraints 
A = [9   1 -9   -2 1 0 0
     1 1/3 -2 -1/3 0 1 0
    -9  -1  9    2 0 0 1]
# b will contain the amount of resources
b = [0, 0, 1]
# c will contain coefficients of objective function -z
c = [3, -1, 6, 0, 0, 0, 0]
# β will contain the initial basic variable indices
βi = [5, 6, 7]

println("Initial basic variable indices: ", βi)

# Call the Simplex-function 6 times with no anti-cycling rules
Simplex(A, b, c, βi, Print = true, Revised = true);


Initial basic variable indices: [5, 6, 7]
Iteration 1
    Reduced costs: [3.0 -1.0 6.0 0.0 0.0 0.0 0.0]
    Entering variable index: 2
    Min ratio test x_β/d: [0.0, 0.0, Inf]
    Leaving variable index: 5
    Basic variable indices: [2, 6, 7]
Iteration 2
    Reduced costs: [12.0 0.0 -3.0 -2.0 1.0 0.0 0.0]
    Entering variable index: 3
    Min ratio test x_β/d: [Inf, 0.0, Inf]
    Leaving variable index: 6
    Basic variable indices: [2, 3, 7]
Iteration 3
    Reduced costs: [6.0 0.0 0.0 -1.0 0.0 3.0 0.0]
    Entering variable index: 4
    Min ratio test x_β/d: [0.0, 0.0, Inf]
    Leaving variable index: 2
    Basic variable indices: [4, 3, 7]
Iteration 4
    Reduced costs: [-3.0 1.0 0.0 0.0 -2.0 12.0 0.0]
    Entering variable index: 1
    Min ratio test x_β/d: [Inf, 0.0, Inf]
    Leaving variable index: 3
    Basic variable indices: [4, 1, 7]
Iteration 5
    Reduced costs: [0.0 0.0 3.0 0.0 -1.0 6.0 0.0]
    Entering variable index: 5
    Min ratio test x_β/d: [0.0, 0.0, 1.0]
    Lea

In [4]:
# Call the Simplex-function using Bland's rule
Simplex(A, b, c, βi, Bland = true, Print = true, Revised = true);

Iteration 1
    Reduced costs: [3.0 -1.0 6.0 0.0 0.0 0.0 0.0]
    Entering variable index: 2
    Min ratio test x_β/d: [0.0, 0.0, Inf]
    Leaving variable index: 5
    Basic variable indices: [2, 6, 7]
Iteration 2
    Reduced costs: [12.0 0.0 -3.0 -2.0 1.0 0.0 0.0]
    Entering variable index: 3
    Min ratio test x_β/d: [Inf, 0.0, Inf]
    Leaving variable index: 6
    Basic variable indices: [2, 3, 7]
Iteration 3
    Reduced costs: [6.0 0.0 0.0 -1.0 0.0 3.0 0.0]
    Entering variable index: 4
    Min ratio test x_β/d: [0.0, 0.0, Inf]
    Leaving variable index: 2
    Basic variable indices: [4, 3, 7]
Iteration 4
    Reduced costs: [-3.0 1.0 0.0 0.0 -2.0 12.0 0.0]
    Entering variable index: 1
    Min ratio test x_β/d: [Inf, 0.0, Inf]
    Leaving variable index: 3
    Basic variable indices: [4, 1, 7]
Iteration 5
    Reduced costs: [0.0 0.0 3.0 0.0 -1.0 6.0 0.0]
    Entering variable index: 5
    Min ratio test x_β/d: [0.0, 0.0, 1.0]
    Leaving variable index: 1
    Basic variable 

### Test problem for Dual Simplex

In [5]:
# A will contain the coefficients of the constraints 
A = [2 1 -1  0 0
     1 0  0 -1 0
     2 3  0  0 1]
# b will contain the amount of resources
b = [5, 1, 6]
# c will contain coefficients of objective function -z
c = [5, 1, 0, 0, 0]
# β will contain the initial basic variable indices
βi = [3, 4, 5]

# Call the Simplex-function on the dual using Bland's rule
DualSimplex(A, b, c, βi, Bland = true, Revised = true, Print = true);


Iteration 1
    Current solution: [0.0, 0.0, -5.0, -1.0, 6.0]
    Leaving variable index: 3
    Min ratio test cₙ/-aₙ: [5.0, 1.0] / [2.0, 1.0]
    Entering variable index: 2
    Basic variable indices: [2, 4, 5]
Iteration 2
    Current solution: [0.0, 5.0, 0.0, -1.0, -9.0]
    Leaving variable index: 5
    Min ratio test cₙ/-aₙ: [3.0, 1.0] / [4.0, -3.0]
    Entering variable index: 1
    Basic variable indices: [2, 4, 1]
Optimum reached!
    The optimal primal solution: [2.25, 0.5, 0.0, 1.25, 0.0]
    The optimal dual solution: [3.25, -0.0, -0.75]
    The optimal cost: 11.75


### Bencmarking of the Bartels-Golub and the naive basis updates on randomly generated linear problems

In [8]:
using TimerOutputs

# Timer for generic solver
function time(n, timerOutput, header, A, b, c, βi, Rev)
    for i in 1:n
        # Construct a random LP of form Ax <= b
        # Dimensions
        n = rand(10:100)
        m = rand(100:1000)

        # A basic solution
        v_β = rand(n, 1)
        βv = rand(collect(1:m), n, 1)
        v = zeros(m)
        v[βv] = v_β
        
        # Cost vector
        c = rand(m, 1)
        
        # Coefficient matrix
        A = rand(n, m)
        
        #Initial basis
        I_n = Matrix{Float64}(I, n, n)
        A[:, (m-n+1):(m)] = I_n
        βi =  collect((m-n+1):(m))

        # Slack
        δ = rand(n, 1)
        
        # Constraint vector
        b = A*v+δ
        
        @timeit to header Simplex(A, b, c, βi, Print = false, Revised = Rev, ϵ=10^(-12));
    end
    return to
end

time (generic function with 1 method)

In [9]:
#### TIME ####
to = TimerOutput()
n = 100

to = time(n, to, "Naive", A, b, c, βi, false)
to = time(n, to, "Revised", A, b, c, βi, true)

to

 ──────────────────────────────────────────────────────────────────
                           Time                   Allocations      
                   ──────────────────────   ───────────────────────
 Tot / % measured:      59.0s / 97.3%           9.22GiB / 96.6%    

 Section   ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────
 Naive        100    31.7s  55.2%   317ms   4.47GiB  50.2%  45.8MiB
 Revised      100    25.7s  44.8%   257ms   4.43GiB  49.8%  45.4MiB
 ──────────────────────────────────────────────────────────────────